In [1]:
import numpy as np
from montlake.vendor.tfcompat.hparam import HParams
from montlake.exec.run_exp import subset_data

import itertools
import numpy as np
import scipy
import json
# export 

from montlake.atomgeom.features import get_features,get_D_feats_feats
from montlake.atomgeom.utils import get_atoms_4
from montlake.simulations.rigidethanol import get_rigid_ethanol_data
from montlake.utils.utils import get_234_indices, get_atoms3_full, get_atoms4_full, data_stream_custom_range, get_cosines
from montlake.geometry.geometry import get_geom, get_wlpca_tangent_sel, get_rm_tangent_sel
from montlake.gradients.estimate import get_grads_pullback
from montlake.statistics.normalization import normalize_L212
from montlake.optimization.gradientgrouplasso import get_sr_lambda_parallel
from montlake.optimization.utils import get_selected_function_ids,get_selected_functions_lm2
from montlake.utils.replicates import Replicate, get_supports_brute,get_supports_lasso

from megaman.embedding import SpectralEmbedding

import dill as pickle
import os
import sys
import numpy as np
import itertools
from itertools import permutations,combinations
from sklearn.decomposition import TruncatedSVD
import pathos
from pathos.multiprocessing import ProcessingPool as Pool

In [2]:

config_file = '/Users/samsonkoelle/tunatostada/experiments/configs/malonaldehyde_full.json'

config = {}
with open(config_file) as f:
    config.update(json.load(f))

args = {}
args['outdir'] = '/Users/samsonkoelle/'
args['raw_data'] = '/Users/samsonkoelle/thesis_data/raw_data/malonaldehyde.mat'
args['nreps'] = 1
args['mflasso'] = True
args['name'] = "mal_full_mf"
config.update(args)

hparams = HParams(**config)
#    if hparams.data_sub == None:
positions, randomindices = subset_data(hparams)

#angle_indices = np.load('/Users/samsonkoelle/thesis_data/raw_data/malonaldehydeindices022119.npy')
#positions_all = scipy.io.loadmat(hparams.raw_data)
#positions = positions_all['R'][angle_indices]
hparams.radius = 3.545

50000  points avaiable


In [3]:
    d = hparams.d
    n_components = hparams.n_components
    atoms2_feat = hparams.atoms2_feat
    atoms3_feat = hparams.atoms3_feat
    atoms4_feat = hparams.atoms4_feat
    atoms2_dict = hparams.atoms2_dict
    atoms3_dict = hparams.atoms3_dict
    atoms4_dict = hparams.atoms4_dict
    diagram = hparams.diagram

    ii = np.asarray(hparams.ii)
    jj = np.asarray(hparams.jj)
    outfile = hparams.outdir + '/' + hparams.name + 'results_mflasso'
    print('loading geometric features')
    natoms = positions.shape[1]
    n = positions.shape[0]
    atoms2 = np.asarray(list(itertools.combinations(range(natoms), 2)))
    atoms2full = atoms2
    atoms3 = np.asarray(list(itertools.combinations(range(natoms), 3)))
    atoms4 = np.asarray(list(itertools.combinations(range(natoms), 4)))
    atoms3full = get_atoms3_full(atoms3)
    atoms4full = get_atoms4_full(atoms4)

    if atoms2_feat:
        atoms2_feats = atoms2full
    else:
        atoms2_feats = np.asarray([])

    if atoms3_feat:
        atoms3_feats = atoms3full
    else:
        atoms3_feats = np.asarray([])

    if atoms4_feat:
        atoms4_feats = atoms4full
    else:
        atoms4_feats = np.asarray([])

    print('computing featurization')
    cores = pathos.multiprocessing.cpu_count() - 1
    pool = Pool(cores)
    print('feature dimensions 234',atoms2_feats.shape, atoms3_feats.shape,atoms4_feats.shape)
    #pool.close()
    #pool.restart()
    results = pool.map(lambda i: get_features(positions[i],
                               atoms2 = atoms2_feats,
                               atoms3 = atoms3_feats,
                               atoms4 = atoms4_feats),
        data_stream_custom_range(list(range(n))))
    data = np.vstack([np.hstack(results[i]) for i in range(n)])
    #data = data - np.mean(data, axis = 0)
    svd = TruncatedSVD(n_components=50)
    data_svd = svd.fit_transform(data)

    print('computing geometry')
    radius = hparams.radius
    n_neighbors = hparams.n_neighbors
    geom = get_geom(data_svd, radius, n_neighbors)

    print('computing embedding')
    spectral_embedding = SpectralEmbedding(n_components=n_components,eigen_solver='arpack',geom=geom)
    embed_spectral = spectral_embedding.fit_transform(data_svd)

    print('getting gradients')
    if atoms2_dict:
        atoms2_dicts = atoms2full
    else:
        atoms2_dicts = np.asarray([])
    if atoms3_dict:
        atoms3_dicts = atoms3full
    else:
        atoms3_dicts = np.asarray([])
    atoms4_dicts= get_atoms_4(natoms, ii, jj)[0]
    
    p = len(atoms2_dicts) + len(atoms3_dicts) + len(atoms4_dicts)
    replicates = {}
    embedding = embed_spectral
    nreps = 1


loading geometric features
computing featurization
feature dimensions 234 (0,) (252, 3) (0,)
computing geometry
computing embedding


/Users/samsonkoelle/miniconda3/envs/manifold_env_april/lib/python3.5/site-packages/sklearn/utils/validation.py:457: DeprecationWarning: Passing 'None' to parameter 'accept_sparse' in methods check_array and check_X_y is deprecated in version 0.19 and will be removed in 0.21. Use 'accept_sparse=False'  instead.
  " instead.", DeprecationWarning)


getting gradients


In [7]:
embed_spectral

(50000, 3)

In [ ]:
alpha = 1
s = 1
title = 'Malonaldehyde'
n = 50000
colors = np.vstack([np.hstack(results['selected_function_values'][i]) for i in range(n)])
c= colors[:,0]
embed = results['embed']
plot_manifold_3d(embed_spectral, s, alpha, c, title)#from montlake.plotting.manifold import plot_manifold

In [4]:
    nsel = 100#hparams.nsel 32413
    for r in range(nreps):
        replicates[r] = Replicate(nsel = 100, n = 50000)
        #replicates[r] = Replicate(selected_points = [32413], n = 50000)
        replicates[r].tangent_bases_M = get_wlpca_tangent_sel(data_svd, geom, replicates[r].selected_points, d)
        D_feats_feats = np.asarray([get_D_feats_feats(positions[replicates[r].selected_points[i]],
                   atoms2in = atoms2_feats,
                   atoms3in = atoms3_feats,
                   atoms4in = atoms4_feats,
                   atoms2out = atoms2_dicts,
                   atoms3out = atoms3_dicts,
                   atoms4out = atoms4_dicts) for i in range(nsel)])
        replicates[r].dg_x = np.asarray([svd.transform(D_feats_feats[i].transpose()).transpose() for i in range(nsel)])
        replicates[r].dg_x_normalized = normalize_L212(replicates[r].dg_x)
        replicates[r].dg_M = np.einsum('i b p, i b d -> i d p', replicates[r].dg_x_normalized, replicates[r].tangent_bases_M)


In [5]:
    cosine = get_cosines(replicates[0].dg_M)
    cosine_abs = np.mean(np.abs(cosine), axis = 0)
    

In [6]:
cosine_abs

array([[1.        , 0.99645779, 0.99472244, 0.99073448, 0.99484645,
        0.99129501, 0.16500473, 0.1751039 , 0.17067533, 0.15845808,
        0.16862049, 0.16658426],
       [0.99645779, 1.        , 0.99342787, 0.99546294, 0.99285749,
        0.99554561, 0.15450148, 0.15805404, 0.15068959, 0.14707917,
        0.15173088, 0.15027997],
       [0.99472244, 0.99342787, 1.        , 0.99714028, 0.99788656,
        0.99562929, 0.1646736 , 0.18996258, 0.18073072, 0.15840727,
        0.18174611, 0.17847386],
       [0.99073448, 0.99546294, 0.99714028, 1.        , 0.994486  ,
        0.99807572, 0.16229515, 0.18736715, 0.17356945, 0.15392666,
        0.1755635 , 0.1692167 ],
       [0.99484645, 0.99285749, 0.99788656, 0.994486  , 1.        ,
        0.9969083 , 0.17076235, 0.19484138, 0.18935942, 0.16763937,
        0.19063195, 0.19057273],
       [0.99129501, 0.99554561, 0.99562929, 0.99807572, 0.9969083 ,
        1.        , 0.16233513, 0.18610413, 0.1773626 , 0.15662977,
        0.17728825,

In [6]:
cosine_abs

array([[1.        , 0.99068505, 0.99524824, 0.98657156, 0.99418384,
        0.98443742, 0.17903933, 0.19188834, 0.19089276, 0.17465749,
        0.18816307, 0.18810075],
       [0.99068505, 1.        , 0.98943639, 0.99591385, 0.98993213,
        0.99542733, 0.15921578, 0.16479838, 0.17035501, 0.15619027,
        0.16083618, 0.16613559],
       [0.99524824, 0.98943639, 1.        , 0.99345119, 0.99740612,
        0.98958804, 0.16756434, 0.19962498, 0.19490434, 0.16547116,
        0.19657793, 0.19174497],
       [0.98657156, 0.99591385, 0.99345119, 1.        , 0.99204597,
        0.99740325, 0.15522126, 0.17585005, 0.17897211, 0.15397652,
        0.1720543 , 0.17344077],
       [0.99418384, 0.98993213, 0.99740612, 0.99204597, 1.        ,
        0.99321789, 0.17197407, 0.19940124, 0.19337385, 0.1724485 ,
        0.19452915, 0.18970755],
       [0.98443742, 0.99542733, 0.98958804, 0.99740325, 0.99321789,
        1.        , 0.16129063, 0.17534108, 0.17837053, 0.16400831,
        0.17337653,

In [9]:
cosine_abs

array([[1.        , 0.99665977, 0.99676816, 0.99293466, 0.99692968,
        0.99301677, 0.16508702, 0.17118126, 0.17304738, 0.16009655,
        0.17291215, 0.17597352],
       [0.99665977, 1.        , 0.99374891, 0.99653378, 0.993998  ,
        0.99684432, 0.16465292, 0.17124612, 0.17521698, 0.1594505 ,
        0.17175456, 0.17319489],
       [0.99676816, 0.99374891, 1.        , 0.99655121, 0.99786316,
        0.99422316, 0.15418957, 0.17009788, 0.17665551, 0.15145762,
        0.16996545, 0.17833666],
       [0.99293466, 0.99653378, 0.99655121, 1.        , 0.99450819,
        0.99766771, 0.16058522, 0.1800734 , 0.18705669, 0.15410762,
        0.17426175, 0.18082419],
       [0.99692968, 0.993998  , 0.99786316, 0.99450819, 1.        ,
        0.99645226, 0.16373543, 0.18070637, 0.18714089, 0.16159465,
        0.18018454, 0.18902271],
       [0.99301677, 0.99684432, 0.99422316, 0.99766771, 0.99645226,
        1.        , 0.16927137, 0.18774086, 0.1934368 , 0.16282352,
        0.18392293,